In [ ]:
from sqlalchemy import create_engine
# 접속 정보
host = "10.28.224.177"
port = 30634
database = "postgres"
username = "postgres"
password = "0104"

# SQLAlchemy 엔진 생성
engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}")

query = f'SELECT * FROM public."Comments"'
comments_df = pd.read_sql(query, engine)

In [13]:
import pandas as pd
import asyncpg

# db 접속하기
conn = await asyncpg.connect(
        host="10.28.224.177",
        port="30634",  
        user="postgres",
        password="0104",
        database="postgres"
        )
query = f"""
    SELECT "viewCount", "Donation",
        (SELECT AVG(CAST("viewCount" as float)) FROM "Channel") as avg_viewcount,
        (SELECT AVG(CAST("Donation" as float)) FROM "Channel") as avg_donation
    FROM public."Channel"
    WHERE "name" = '피식대학'
    """

rows = await conn.fetch(query)
df = pd.DataFrame(rows, columns=[key for key in rows[0].keys()])
print(df)
#print(len(df))
#print(df.columns)
print(df['avg_Donation'])
print(type(df.iloc[0]['channel_id']))
print('-------------------------------------------------')
view_profit_user = (df['viewCount'])*2, int(df['viewCount']*4.5)
view_profit_avg = (df['avg_viewCount']*2, int(df['avg_viewCount']*4.5))
donation_profit_user = df['Donation']
donation_profit_avg = df['avg_Donation']
#print(df.iloc[0]["videoDescription"])
await conn.close()

    viewCount Donation  avg_viewcount avg_donation
0  2179523566     None    819335238.3         None


KeyError: 'avg_Donation'

# table 아예 지우고 덮어쓰기

In [ ]:
import asyncpg
import pandas as pd
import numpy as np

async def overwrite_table_with_csv(file_path, table_name):
    # CSV 파일 읽기
    data = pd.read_csv(file_path)
    

    '''# Ensure proper data types
    for col in ['dailySubscriberCount', 'dailyViewCount', 'totalSubscriberCount', 'totalViewCount']:
        if col in data.columns:
            data[col] = data[col].astype('Int64')  # Nullable integer type'''
    data = data.replace({pd.NA: None, np.nan: None})

    # Convert rows to tuples
    rows = [tuple(row) for row in data.itertuples(index=False, name=None)]
    
    # PostgreSQL 연결
    conn = await asyncpg.connect(
    		host="10.28.224.177",
            port="30634",  
            user="postgres",
            password="0104",
            database="postgres"
            )

    try:
        await conn.execute(f"TRUNCATE TABLE {table_name};")
        print(f"Table {table_name} truncated.")
        
        # 데이터 삽입
        columns = ['"'+column+'"' for column in list(data.columns)]
        print(*columns)
        values_placeholder = ", ".join([f"${i+1}" for i in range(len(columns))])
        insert_query = f"""INSERT INTO "{table_name}" ({', '.join(columns)}) VALUES ({values_placeholder});"""

        # 데이터 행 생성
        rows = [tuple(row) for row in data.itertuples(index=False, name=None)]
        await conn.executemany(insert_query, rows)
        print(f"""{len(rows)} rows inserted into "{table_name}".""")

    except Exception as e:
        print(f"Error: {e}")
    
    finally:
        await conn.close()

# 실행 코드
import asyncio

async def main():
    csv_path = "_Channel__202501271549.csv"  # CSV 파일 경로
    table_name = "Channel"         # 테이블 이름
    await overwrite_table_with_csv(csv_path, table_name)

asyncio.run(main())


In [20]:
import psycopg2
import pandas as pd
import numpy as np

def overwrite_table_with_csv(file_path, table_name):
    # CSV 파일 읽기
    data = pd.read_csv(file_path)

    # NaN 값을 None으로 변환 (PostgreSQL에서 NULL로 처리)
    data = data.replace({pd.NA: None, np.nan: None})

    # 데이터 행을 튜플 형태로 변환
    rows = [tuple(row) for row in data.itertuples(index=False, name=None)]

    # PostgreSQL 연결
    conn = psycopg2.connect(
        host="10.28.224.177",
        port="30634",
        user="postgres",
        password="0104",
        database="postgres"
    )

    try:
        with conn.cursor() as cur:
            # 테이블 비우기 (TRUNCATE)
            cur.execute(f"TRUNCATE TABLE {table_name};")
            print(f"Table {table_name} truncated.")

            # 데이터 삽입 쿼리 생성
            columns = ['"'+column+'"' for column in list(data.columns)]
            values_placeholder = ", ".join(["%s"] * len(columns))
            insert_query = f"""INSERT INTO "{table_name}" ({', '.join(columns)}) VALUES ({values_placeholder});"""

            # 데이터 삽입 실행
            cur.executemany(insert_query, rows)
            print(f"{len(rows)} rows inserted into '{table_name}'.")

        # 변경사항 커밋
        conn.commit()

    except Exception as e:
        print(f"Error: {e}")
        conn.rollback()

    finally:
        conn.close()

# 실행 코드
if __name__ == "__main__":
    csv_path = "_Channel__202501271549.csv"  # CSV 파일 경로
    table_name = "Channel"                  # 테이블 이름
    overwrite_table_with_csv(csv_path, table_name)


Error: relation "channel" does not exist



# 기존 row에 데이터 추가 (UPDATE)

In [97]:
import asyncpg
import pandas as pd
import numpy as np

async def overwrite_table_with_csv(file_path, table_name, columns):
    # CSV 파일 읽기
    data = pd.read_csv(file_path)
    rows = [tuple(row) for row in data[['id'] + columns].itertuples(index=False, name=None)]
    print(rows)
    # PostgreSQL 연결
    conn = await asyncpg.connect(
    		host="10.28.224.177",
            port="30634",  
            user="postgres",
            password="0104",
            database="postgres"
            )

    try:
        # 데이터 삽입
        columns = ['"'+column+'"' for column in columns]
        print(*columns)
        values_placeholder = ", ".join([f"{col} = ${i+2}" for i, col in enumerate(columns)])
        insert_query = f"""
        UPDATE "{table_name}"
        SET {values_placeholder}
        WHERE "id" = $1;
        """ 

        await conn.executemany(insert_query, rows)
        print(f"""{len(rows)} rows inserted into "{table_name}".""")

    except Exception as e:
        print(f"Error: {e}")
    
    finally:
        await conn.close()

# 실행 코드
import asyncio

async def main():
    csv_path = "라이브 영상 수.csv"  # CSV 파일 경로
    table_name = "Channel"         # 테이블 이름
    columns = ["NumLiveBroadcasting", "AverageViewerCount"]
    await overwrite_table_with_csv(csv_path, table_name, columns)

asyncio.run(main())


[(0, 20, 1242), (1, 0, 0), (2, 60, 2713), (3, 660, 241), (4, 24, 957), (5, 2, 2288), (6, 0, 0), (7, 40, 8800), (8, 2, 2723), (9, 0, 0)]
"NumLiveBroadcasting" "AverageViewerCount"
10 rows inserted into "Channel".


In [19]:
import psycopg2
import pandas as pd

def overwrite_table_with_csv(file_path, table_name,):
    # CSV 파일 읽기
    data = pd.read_csv(file_path)
    columns = data.columns
    rows = [tuple(row) for row in data[['id'] + columns].itertuples(index=False, name=None)]
    
    # PostgreSQL 연결
    conn = psycopg2.connect(
        host="10.28.224.177",
        port="30634",
        user="postgres",
        password="0104",
        database="postgres"
    )
    try:
        with conn.cursor() as cur:
            # 정렬된 새 테이블 생성
            create_query = f"""
            CREATE TABLE Channel_Sorted AS
            SELECT *
            FROM {table_name}
            ORDER BY id;
            
            -- 기존 테이블 삭제
            DROP TABLE {table_name};
            
            -- 새 테이블 이름 변경
            ALTER TABLE Channel_Sorted RENAME TO {table_name};
            """
            cur.execute(create_query)

            # 데이터 삽입
            columns_str = ", ".join(['"id"'] + [f'"{col}"' for col in columns])
            values_placeholder = ", ".join(["%s"] * len(columns))
            
            insert_query = f"""
            INSERT INTO {table_name} ({columns_str})
            VALUES ({values_placeholder});
            """
            
            cur.executemany(insert_query, rows)
            
        conn.commit()
        print(f"{len(rows)} rows inserted into '{table_name}'.")

    except Exception as e:
        print(f"Error: {e}")
        conn.rollback()
    
    finally:
        conn.close()

# 실행 코드
if __name__ == "__main__":
    csv_path = "_Channel__202501271549.csv"  # CSV 파일 경로
    table_name = "Channel"         # 테이블 이름
    overwrite_table_with_csv(csv_path, table_name)


RuntimeError: asyncio.run() cannot be called from a running event loop